 Importing our dependencies

In [ ]:
import keras
import numpy as np
import random
import io

Using TensorFlow backend.


 Loading the dataset

In [ ]:
 path = keras.utils.data_utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print(len(text))

606208/600901 [==============================] - 0s 1us/step
600893


 Assigning Numerical values to characters

In [ ]:
 chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


 Next, we create multidimensional arrays out of the text data that we inputed so that the entire corpus of text is enumerated (is assigned a number instead of a character). This will be used for training and makes everything more coherent.

In [ ]:
 sentences = []
next_chars = []
for i in range(0, len(text) - 40, 3):
    sentences.append(text[i: i + 40])
    next_chars.append(text[i + 40])
 
x = np.zeros((len(sentences), 40, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

 model with an LSTM layer

In [ ]:
 model = keras.models.Sequential()
model.add(keras.layers.LSTM(128, input_shape=(40, len(chars))))
model.add(keras.layers.Dense(len(chars), activation='softmax'))
 
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

 This function will compare the sample predictions with the original text. The temperature represents how accurate or confident the model's predictions will be.

In [ ]:
 def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Training and testing the model

In [ ]:
 model.fit(x, y, batch_size=18, epochs=20)
 
 
#generating random text
start_index = random.randint(0, len(text) - 40 - 1)
for diversity in [0.2, 0.5, 1.0, 1.2]:
    generated = ''
    sentence = text[start_index: start_index + 40]
    generated += sentence
 
    for i in range(400):
        x_pred = np.zeros((1, 40, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.
 
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
 
        generated += next_char
        sentence = sentence[1:] + next_char
with open('example.txt', 'w') as f:
    f.write(generated)

Epoch 1/20
200285/200285 [==============================] - 506s 3ms/step - loss: 1.9729
Epoch 2/20
200285/200285 [==============================] - 545s 3ms/step - loss: 1.8051
Epoch 3/20
200285/200285 [==============================] - 548s 3ms/step - loss: 1.7821
Epoch 4/20
200285/200285 [==============================] - 574s 3ms/step - loss: 1.7995
Epoch 5/20
200285/200285 [==============================] - 581s 3ms/step - loss: 1.8445
Epoch 6/20
168588/200285 [========================>.....] - ETA: 1:29 - loss: 1.9966

Saving the sample to our Computer(optional)

In [ ]:
 from google.colab import files
files.download('example.txt')